# Working with PennyLane (Qubit Rotation)
**By:** Arjun Bhamra

### Imports

In [2]:
import pennylane as qml
from pennylane import numpy as np

### First Test

In [6]:
#initializing the circuit
def my_quantum_function(x, y):
    qml.RZ(x, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(y, wires=1)
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))#for multiple measurements

dev = qml.device('default.qubit', wires=2, shots=1000, analytic=False)

In [7]:
#creating a qnode to work with
circuit = qml.QNode(my_quantum_function, dev)

In [8]:
print(circuit(np.pi/4, 0.7))

[1.   0.77]


In [9]:
print(circuit.draw())

 0: ──RZ(0.785)──╭C───────────┤ ⟨Z⟩ 
 1: ─────────────╰X──RY(0.7)──┤ ⟨Z⟩ 



### Qubit Rotation

To see how PennyLane allows the easy construction and optimization of quantum functions, let’s consider the simple case of qubit rotation the PennyLane version of the ‘Hello, world!’ example.

The task at hand is to optimize two rotation gates in order to flip a single qubit from state $|0\rangle$ to state $|1\rangle$.

The circuit should look like:
$|0\rangle - R_x(\phi_1)-R_y(\phi_2)-[meas]-|1\rangle$, where $[meas]$ is the measurement operator. We're looking to optimize $\phi_1$ and $\phi_2$.

We will be following [this](https://pennylane.ai/qml/demos/tutorial_qubit_rotation.html) tutorial from the PennyLane website.

### Doing the math

**Step 1:** Initialize State $|0\rangle$

**Step 2:** Apply the rotation gate $$R_x(\phi_1) = e^{-i\phi_1\frac{\sigma_x}{2}} = \begin{bmatrix}\cos{\frac{\phi_1}{2}} && -i\sin{\frac{\phi_1}{2}} \\ -i\sin{\frac{\phi_1}{2}} && \cos{\frac{\phi_1}{2}}\end{bmatrix}$$

**Step 3:** Apply the rotation gate $$R_y(\phi_2) = e^{-i\phi_2\frac{\sigma_x}{2}} = \begin{bmatrix}\cos{\frac{\phi_2}{2}} && -i\sin{\frac{\phi_2}{2}} \\ -i\sin{\frac{\phi_2}{2}} && \cos{\frac{\phi_2}{2}}\end{bmatrix}$$

At this point, the qubit is now in the state $$|\psi\rangle = R_y(\phi_2)R_x(\phi_1)|0\rangle$$

Finally, measure the state by finding the **expectation value** with respect to the $Z$-basis using $$\langle\psi|\sigma_z|\psi\rangle = \langle0∣R_x(\phi_1)^{\dagger}R_y(\phi_2)^{\dagger}\sigma_zR_y(\phi_2)R_x(\phi_1)∣0\rangle=cos(\phi_1)cos(\phi_2)$$

Depending on the circuit parameters $\phi_1$ and $\phi_2$, the output expectation lies between $1$ (if $|\psi\rangle=|0\rangle$) and $−1$ (if $|\psi\rangle=|1\rangle$).

### Making the Qubit Rotation Circuit with PennyLane

#### Imports

In [6]:
import pennylane as qml
from pennylane import numpy as np

#### PennyLane: Devices
**Device:** Any computational object that can apply quantum operations and return an measurement value is called a quantum device.

In [7]:
dev1 = qml.device("default.qubit", wires=1)
#default qubit refers to the type of model used, and wires is obviously the number of qubits

#### PennyLane: QNodes
**QNode:** QNodes are an abstract encapsulation of a quantum function, described by a quantum circuit. QNodes are bound to a particular quantum device, which is used to evaluate expectation and variance values of this circuit. 

You can create QNodes with the `QNode` class or the `qnode()` decorator

#### Creating a Circuit Function
We create a circuit function first so we can evaluate it in the `QNode`. Here, our circuit is going to be $|\psi\rangle = R_y(\phi_2)R_x(\phi_1)|0\rangle$ as seen above.

**Note:** `params`, as seen below, can be a tuple, list, or array that takes in a series of parameters, and we use the individual elements as gate parameters (see code).

**Note:** We must always specify the subsystem the operation applies to, by passing the `wires` argument; this may be a *list* or an *integer*, depending on how many wires the operation acts on.

In [8]:
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    #applying both gates to first (and only) wire, giving them one of the two params respectively
    return qml.expval(qml.PauliZ(0))
    #finding the expectation value/measurement of the circuit

#### Converting to a QNode and running

In [9]:
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

In [10]:
print(circuit([0.54, 0.12]))
print(circuit.draw())

0.8515405859048366
 0: ──RX(0.54)──RY(0.12)──┤ ⟨Z⟩ 



#### Calculating Quantum Gradients
We used the `grad()` function to calculate the gradients of the circuit.
This returns another function, representing the gradient (**i.e., the vector of partial derivatives**) of `circuit`. 

In this case, the function circuit takes one argument (`params`), so we specify `argnum=0`. Because the argument has two elements, the returned gradient is two-dimensional. We can then evaluate this gradient function at any point in the parameter space.

In [11]:
dcircuit = qml.grad(circuit, argnum=0)

In [12]:
print(dcircuit([0.54, 0.12]))

[array(-0.51043865), array(-0.1026782)]


**A Note On Arguments:**

Quantum circuit functions, being a restricted subset of Python functions, can also make use of multiple positional arguments and keyword arguments. For example, we could have defined the above quantum circuit function using two positional arguments, instead of one array argument:

In [13]:
@qml.qnode(dev1)
def circuit2(phi1, phi2):
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))

In [14]:
dcircuit = qml.grad(circuit2, argnum=[0, 1])
print(dcircuit(0.54, 0.12))

(array(-0.51043865), array(-0.1026782))


#### Optimization of the Parameters

Use PennyLane’s built-in optimizers to optimize the two circuit parameters $\phi_1$ and $\phi_2$ such that the qubit, originally in state $|0\rangle$, is rotated to be in state $|1\rangle$. This is equivalent to measuring a Pauli-Z expectation value of $−1$, since the state $|1\rangle$ is an eigenvector of the Pauli-Z matrix with eigenvalue $\lambda=−1$.

To do so, we need to define a **cost** function. By minimizing the cost function, the optimizer will determine the values of the circuit parameters that produce the desired outcome.

In this case, our desired outcome is a Pauli-Z expectation value of $−1$. Since we know that the Pauli-Z expectation is bound between $[−1,1]$, we can define our cost directly as the output of the QNode:

In [15]:
def cost(x):
    return circuit(x)

In [16]:
#test
init_params = np.array([0.011, 0.012])
print(cost(init_params))

0.9998675058299389


In [17]:
#make the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4)

#num of steps/iterations
steps=100

params = init_params

#optimize
for i in range(steps):
    params = opt.step(cost, params)
    #the step function takes in an objective function (that calculates the cost) and the parameters, operating on 
    #a step by step iterative basis
    
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))
    #print the values of the parameters every 5 steps
    
print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.9961778
Cost after step    10:  0.8974944
Cost after step    15:  0.1440490
Cost after step    20: -0.1536720
Cost after step    25: -0.9152496
Cost after step    30: -0.9994046
Cost after step    35: -0.9999964
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [7.15266381e-18 3.14159265e+00]


### Yay it works!
We now know that one possible combination of $\phi_1$ and $\phi_2$ to achieve the expectation value of $-1$ (based on the prev. equation $\langle\psi|\sigma_z|\psi\rangle$) is $\phi_1=0$ and $\phi_2=\pi$